In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
import fastimgproto.gridder.conv_funcs as kfuncs
from fastimgproto.gridder.gridder import convolve_to_grid

In [ ]:
%matplotlib inline
# Plot image pixels in cartesian ordering (i.e. y-positive == upwards):
plt.rcParams['image.origin'] = 'lower'
# Make plots bigger
plt.rcParams['figure.figsize'] = [10., 8.]

In [ ]:
plt.rcParams['figure.figsize']

In [ ]:
from fastimgproto.fixtures.data import simple_vis_npz_filepath
with open(simple_vis_npz_filepath, 'rb') as f:
    loaded = dict(np.load(f).items())

In [ ]:
uvw_in_wavelengths = loaded['uvw_lambda']
vis = loaded['vis']

In [ ]:
image_size = 1024 * u.pixel
cell_size = 1 * u.arcsecond
image_size = int(image_size.to(u.pix).value)
grid_pixel_width_in_wavelengths = 1.0 / (cell_size.to(u.rad) * image_size)

In [ ]:
grid_pixel_width_in_wavelengths

In [ ]:
uvw_in_pixels = (uvw_in_wavelengths / grid_pixel_width_in_wavelengths).value

In [ ]:
uvw_in_pixels

In [ ]:
uv_in_pixels= uvw_in_pixels[:,:2]

In [ ]:
kernel = kfuncs.GaussianSinc(trunc=5.)
vis_grid, sampling_grid = convolve_to_grid(kernel, support=5, 
                          image_size=image_size,
                         uv=uv_in_pixels,
                         vis=vis)

In [ ]:
vis_grid

In [ ]:
def fft_to_image_plane(uv_grid):
    return np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(uv_grid)))

In [ ]:
plt.imshow(np.real(fft_to_image_plane(vis_grid)))

In [ ]:
plt.imshow(np.real(fft_to_image_plane(sampling_grid)))